## This notebook shows some example gremlin queries using nodejs gremlin driver. Driver details are here: https://www.npmjs.com/package/gremlin. 

## You will use PixieDust extension that enables a Jupyter Notebook user to invoke Node.js commands.



### Install pixiedust_node

In [ ]:
!pip install pixiedust_node

### Import pixiedust_node

In [ ]:
import pixiedust_node

### Install gremlin javascript driver

In [ ]:
# install node packages
npm.install('gremlin')
npm.install('uuid')

In [ ]:
%%node 

const { v4: uuidv4 } = require('uuid');
const gremlin = require('gremlin');
const traversal = gremlin.process.AnonymousTraversalSource.traversal;
const DriverRemoteConnection = gremlin.driver.DriverRemoteConnection

const { cardinality: { single } } = gremlin.process;
const __ = gremlin.process.statics;
const column = gremlin.process.column
const direction = gremlin.process.direction
const p = gremlin.process.P
const textp = gremlin.process.TextP
const pick = gremlin.process.pick
const pop = gremlin.process.pop
const order = gremlin.process.order
const scope = gremlin.process.scope
const t = gremlin.process.t

const ClusterEndpoint = "<ClusterEndpoint>";
const Port = "Port";

const g = traversal().withRemote(new DriverRemoteConnection(`wss://${ClusterEndpoint}:${Port}/gremlin`));


### Create Single Vertex

In [ ]:
%%node

var personId = uuidv4();

try {
    const queryPromise =  g.addV('person').property(t.id, personId)
            .property('email','user@domain.com')
            .property(single, 'firstname', 'firstname')
            .property(single, 'lastname', 'lastname')
            .next();
    
    queryPromise.then(data => {
        console.log(data);
    })
}
catch(ex){
    console.log(ex);
}


In [ ]:
%%node

var movieId = uuidv4();

try {
    const queryPromise =  g.addV('movie')
            .property(t.id, movieId)
            .property(single, 'title', 'movietitle')
            .next();
    
    queryPromise.then(data => {
        console.log(JSON.stringify(data));
    })

}
catch(ex){
    console.log(ex);
}


### Add edges between existing vertices

In [ ]:
%%node

try {
    const queryPromise =  g.addE("watched")
            .from_(__.V(personId)).to(__.V(movieId))
            .next();
    
    queryPromise.then(data => {
        console.log(data.value);
    })
}
catch(ex){
    console.log(ex);
}

### Add multiple vertices and edges

In [ ]:
%%node

try {
    const queryPromise =  g.addV('person').property(t.id, uuidv4())
            .property('email', "user@domain.com")
            .property(single, 'firstname', 'firstname')
            .property(single, 'lastname', 'lastname')
            .as("user")
            .addV('movie')
            .property(t.id, uuidv4())
            .property(single, 'title', 'movietitle')
            .as("movie")
            .addE("watched")
            .from_('user').to('movie')
            .next();
    
    queryPromise.then(data => {
        console.log(data);
    })
}
catch(ex){
    console.log(ex);
}


### Read vertex and its attributes

In [ ]:
%%node

try {
    
    const getPersonPromise= g.V().hasLabel("movie").valueMap(true).limit(10).toList();
    
    console.log("\n");
    console.log("id|label|title");
                
    getPersonPromise.then(data => {
        data.map(item => {
            console.log(item.get(t.id) + "|" + item.get(t.label) + "|" + item.get("title"))
        });
    });
    
}
catch(ex){
    console.log(ex);
}

### Read vertex connections

In [ ]:
%%node

try {
    
    const getPersonPromise= g.V(personId).out("watched").valueMap(true).limit(10).toList();
    
    getPersonPromise.then(data => {
        data.forEach(item => {
            console.log(item);
        });
    });
    
}
catch(ex){
    console.log(ex);
}

In [ ]:
%%node

function formatAndPrint(data){
     console.log(data.get("personid").padEnd(20," ") + data.get("movies")[0].get("title"));
}

try {
    
    const getPersonPromise = g.V(personId).project('personid','movies')
                                            .by("email").by(__.out("watched").valueMap("title").fold())
                                            .limit(10).toList();
    console.log("\n");
    console.log("Email".padEnd(20," ") + "Title".padEnd(20," "));
    console.log("".padEnd(50,"_"));
    
    getPersonPromise.then(data => {
        data.forEach(item => { 
            formatAndPrint(item);
        });      
    });  
}
catch(ex){
    console.log(ex);
}